## 1. Load the libraries

In [1]:
# update or install libraries
!pip install gradio
!pip install python-dotenv
!pip install --upgrade openai
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.8 MB/s eta 0:

In [2]:
import gradio as gr
import openai
import IPython.display
import spacy

In [3]:
openai.api_key = #Load API key

#2. Prompt Engineering

In [4]:
import spacy
import openai
import gradio as gr

# Entity extraction function
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

# Summarization function
def generate_text(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",  # GPT-3
        prompt=prompt,
        max_tokens=50,
        temperature=0.7
    )
    return response.choices[0].text.strip()

# Sentiment analysis and entity extraction
def classify_text(text):
    prompt_summary = f"""It's nice that they support English, but the menus are a bit strange in English. I wish I could set the language within the app.
    Explain the above in one sentence:"""

    generated_summary = generate_text(prompt_summary)

    entities = extract_entities(text)

    prompt_classification = f"""Classify the review text into negative or positive.

    Text: {text}

    Sentiment:"""

    generated_classification = generate_text(prompt_classification)

    return entities, generated_classification, generated_summary

# 3. Build an App UI with Gradio

In [6]:
# Gradio interface
app = gr.Interface(
    fn=classify_text,
    inputs=gr.inputs.Textbox(label="Enter a review:"),
    outputs=[
        gr.outputs.Textbox(label="Extracted Entities:"),
        gr.outputs.Textbox(label="Sentiment:"),
        gr.outputs.Textbox(label="Summary:")
    ],
    live=True,
    title="An app to review feedback for a product"
)

# Launch Gradio App
app.launch(share=True)

<ipython-input-6-86f4803b27be>:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-6-86f4803b27be>:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-6-86f4803b27be>:4: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-6-86f4803b27be>:6: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  gr.outputs.Textbox(label="Extracted Entities:"),
<ipython-input-6-86f4803b27be>:7: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components fro

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5f9d920f62dbcfc2c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
